Посчитать retention и частоту пользовательского входа для когорт игроков: iOS/Android, страны
СНГ/Северная Америка. Оценить возможную корреляцию между показателями, сравнить полученные
результаты для разных когорт и сделать выводы.

In [2]:
#retention rate+sessions
df=pd.read_sql(f'''
with data as

         (SELECT  event_time, user_id, platform, 
            min(event_time) filter (where event_name='FirstLaunchApp') OVER(PARTITION BY user_id) as date_install,
            case WHEN country in ('RU', 'AZ', 'AM', 'BY','KZ', 'KG','MD', 'TJ', 'UZ', 'UA', 'TM') then 'CIS'
                 WHEN country in ('CA', 'US') then 'NA' else 'Other' end as region,event_name
            FROM data_viz_1068.project_dataset
            WHERE user_id in
                    (SELECT user_id
                    FROM data_viz_1068.project_dataset
                    WHERE event_name='FirstLaunchApp'))
    SELECT lifetime,
        platform, region,
        cast(users as float)/cast(installs as float) as ret,
        sessions
    FROM
        (SELECT lifetime, count(distinct user_id) as users, platform, region,
        max(count(distinct user_id)) OVER (PARTITION BY platform, region) as installs,
        round(avg(sessions),1) sessions
        FROM
            (SELECT max(extract(day from date_trunc('days',event_time-date_install))) as lifetime,
             platform, region, user_id,count(*) filter (where lower(event_name) like '%%launch%%') as sessions 
            FROM data
            GROUP BY 2,3,4) as l
        WHERE lifetime between '0' and '30' and region !='Other'
        GROUP BY 1,3,4) as q
    ORDER BY 1,2,3
''', con=engine)
df.pivot_table(index = ['lifetime'], columns = ['platform','region'], values = ['ret','sessions']).fillna(0).reset_index()
#df.to_csv('./desktop/df.csv', index=False)

lifetime       ret                               sessions        \
platform            android                 ios            android         
region                  CIS        NA       CIS        NA      CIS    NA   
0             0.0  1.000000  1.000000  1.000000  1.000000      2.1   2.4   
1             1.0  0.099218  0.097194  0.098868  0.101699      6.6   4.8   
2             2.0  0.063411  0.058179  0.051961  0.057078      8.2   6.6   
3             3.0  0.050285  0.047228  0.039628  0.047112      8.7   7.2   
4             4.0  0.033008  0.043121  0.036393  0.033749     10.1   7.7   
5             5.0  0.027024  0.038330  0.026082  0.030125      9.4   8.0   
6             6.0  0.025963  0.024641  0.023453  0.031484     12.3   7.9   
7             7.0  0.022102  0.020534  0.025475  0.024236     14.1  13.2   
8             8.0  0.020172  0.028747  0.023251  0.024236     16.4  14.4   
9             9.0  0.016697  0.026694  0.017388  0.016761     13.7  16.2   
10           10.0  0.014960  0.027379  0.015973  0.017441     16.4  10.6   
11           11.0  0.013898  0.017796  0.015164  0.014723     29.0  25.7   
12           12.0  0.011292  0.016427  0.011929  0.012458     21.6  14.8   
13           13.0  0.011582  0.017796  0.012535  0.015176     25.5  15.8   
14           14.0  0.009941  0.017112  0.013344  0.017441     23.5  35.2   
15           15.0  0.009748  0.013689  0.012333  0.014043     23.2  30.4   
16           16.0  0.008493  0.013689  0.010716  0.010872     30.9  20.4   
17           17.0  0.007625  0.013005  0.009098  0.012231     24.0  18.5   
18           18.0  0.006756  0.007529  0.008492  0.010872     26.7  31.2   
19           19.0  0.007818  0.010267  0.008290  0.007701     32.6  36.3   
20           20.0  0.008107  0.013005  0.005661  0.008154     29.7  32.1   
21           21.0  0.006467  0.008214  0.007885  0.012005     23.7  22.6   
22           22.0  0.006080  0.008898  0.005459  0.012231     28.4  17.4   
23           23.0  0.006177  0.008214  0.006470  0.006795     27.1  43.3   
24           24.0  0.005984  0.009582  0.004852  0.008154     31.3  20.1   
25           25.0  0.004729  0.007529  0.004246  0.006795     27.0  32.1   
26           26.0  0.004633  0.006160  0.004852  0.005436     29.1  22.4   
27           27.0  0.005115  0.006845  0.005257  0.007701     38.3  17.6   
28           28.0  0.005308  0.008898  0.004852  0.009287     57.1  41.0   
29           29.0  0.002992  0.006845  0.004044  0.004530     27.5  15.6   
30           30.0  0.003861  0.008898  0.005055  0.007701     43.6  37.8   

                      
platform   ios        
region     CIS    NA  
0          2.1   2.1  
1          6.0   4.8  
2          6.0   5.6  
3          6.7   6.0  
4          6.6   7.1  
5          8.5   8.5  
6         12.7   8.5  
7         14.2  10.3  
8         11.4   9.7  
9         12.2  12.1  
10        12.4  16.3  
11        13.1  11.7  
12        16.1  12.5  
13        13.7  13.2  
14        18.0  14.8  
15        12.8  13.6  
16        22.6  17.6  
17        23.8  14.5  
18        21.1  21.4  
19        26.9  16.0  
20        22.3  19.3  
21        18.9  16.0  
22        26.3  20.8  
23        17.9  21.5  
24        23.5  20.9  
25        28.4  12.7  
26        17.0  16.6  
27        22.2  12.8  
28        32.4  19.5  
29        21.5  28.5  
30        25.0  58.6

https://docs.google.com/spreadsheets/d/1me9TAVYme6XFdJX3OyvuBfRPzR2PX3XH2nd9lokyZrU/edit?usp=sharing